In [1]:
import pandas as pd
import numpy as np
import locale
import os

In [2]:
# **1. Ursprüngliche Daten laden**

# Überprüfen, ob alle benötigten Dateien vorhanden sind
required_files = ["umsatzdaten_gekuerzt.csv", "kiwo.csv", "wetter.csv", "Data Import and Preparation/feiertage.csv"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Datei '{file}' nicht gefunden.")

# Daten laden
umsatzdaten = pd.read_csv("umsatzdaten_gekuerzt.csv")
kiwo_data = pd.read_csv("kiwo.csv")
wetter_data = pd.read_csv("wetter.csv")
feiertage = pd.read_csv("Data Import and Preparation/feiertage.csv")  # Enthält Spalten 'Datum' und 'Feiertag'

In [3]:
# **2. Daten vorverarbeiten**

# Konvertiere 'Datum' in datetime für alle Datensätze
umsatzdaten['Datum'] = pd.to_datetime(umsatzdaten['Datum'], errors='coerce')
kiwo_data['Datum'] = pd.to_datetime(kiwo_data['Datum'], errors='coerce')
wetter_data['Datum'] = pd.to_datetime(wetter_data['Datum'], errors='coerce')

# Locale auf Deutsch setzen, damit Datumsformate korrekt interpretiert werden
try:
    locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
except locale.Error:
    print("Locale-Einstellung auf 'de_DE.UTF-8' fehlgeschlagen. Weiter ohne Locale.")

# 'Datum' im feiertage-Datensatz anpassen
# Jahr zufällig zwischen 2013 und 2018 setzen
def adjust_year(date_str):
    # Überprüfen, ob der Datumseintrag nicht leer oder NaN ist
    if isinstance(date_str, str) and date_str.strip():  # Prüfen, ob es ein nicht-leerer String ist
        # Punkt und Leerzeichen entfernen, Jahr zufällig wählen
        month_day = date_str.replace('.', '').strip()
        year = np.random.choice([2013, 2014, 2015, 2016, 2017, 2018])  # Zufällig ein Jahr zwischen 2013 und 2018 wählen
        return f"{month_day} {year}"
    else:
        return np.nan  # Rückgabe von NaN, wenn der Eintrag ungültig oder leer ist

# Anwendung der Jahr-Anpassung auf die 'Datum'-Spalte
feiertage['Datum'] = feiertage['Datum'].apply(adjust_year)

# Konvertiere die angepasste 'Datum'-Spalte in datetime
feiertage['Datum'] = pd.to_datetime(feiertage['Datum'], format='%d %b %Y', errors='coerce')

# Zeilen mit nicht parsebaren Datumswerten entfernen
feiertage = feiertage.dropna(subset=['Datum'])

# Debug: Überprüfen der ersten paar Zeilen des feiertage- und umsatzdaten-Datensatzes
print("Feiertage-Datensatz (erste 5 Zeilen):")
print(feiertage.head())

print("\nUmsatzdaten-Datensatz (erste 5 Zeilen):")
print(umsatzdaten.head())

Feiertage-Datensatz (erste 5 Zeilen):
       Datum Feiertag                                               Name  \
2 2014-01-01   Montag                                        Neujahrstag   
3 2016-01-06  Samstag                                Heilige Drei Könige   
4 2014-01-06  Samstag                                Heilige Drei Könige   
5 2016-01-22   Montag                          Deutsch-Französischer Tag   
6 2015-01-27  Samstag  Tag des Gedenkens an die Opfer des Nationalsoz...   

                                Art       Details  
2               Nationaler Feiertag           NaN  
3  Fest- oder Gedenktag, Christlich  NDS*, SN, TH  
4               Regionaler Feiertag    BW, BY, ST  
5              Fest- oder Gedenktag           NaN  
6              Fest- oder Gedenktag           NaN  

Umsatzdaten-Datensatz (erste 5 Zeilen):
       Datum  Warengruppe      Umsatz
0 2013-07-01            1  148.828353
1 2013-07-02            1  159.793757
2 2013-07-03            1  111.885594


In [4]:
# **3. Daten zusammenführen**

# Die Datensätze anhand des 'Datum' zusammenführen
merged_data = umsatzdaten.merge(kiwo_data, on='Datum', how='left')
merged_data = merged_data.merge(wetter_data, on='Datum', how='left')

# Die Feiertagsdaten anhand des 'Datum' hinzufügen, nur die Spalte 'Feiertag' beibehalten
merged_data = merged_data.merge(feiertage[['Datum', 'Name']], on='Datum', how='left')

In [ ]:
# **4. Zusätzliche Variablen erstellen**

# Wochentag hinzufügen (Name des Wochentags)
merged_data['Wochentag'] = merged_data['Datum'].dt.day_name()

# Monat als numerischen Wert hinzufügen
merged_data['Monat'] = merged_data['Datum'].dt.month

# Binäre Feiertags-Flagge hinzufügen (1 für Feiertag, 0 für keinen Feiertag)
merged_data['Ist_Feiertag'] = merged_data['Feiertag'].notnull().astype(int)

# Quartal hinzufügen
merged_data['Quartal'] = merged_data['Datum'].dt.quarter

# Saisonale Variable hinzufügen
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:
        return "Herbst"

merged_data['Saison'] = merged_data['Monat'].apply(get_season)

# Wochenend-Flagge hinzufügen (Samstag/Sonntag)
merged_data['Ist_Wochenende'] = merged_data['Wochentag'].isin(['Saturday', 'Sunday']).astype(int)

In [5]:
# **5. Aufbereitung der Variablen**

# Fehlende Werte behandeln (falls notwendig)
merged_data.fillna(0, inplace=True)

In [6]:

# **6. Exportieren des finalen Datensatzes**

# Erstelle das Ausgabeverzeichnis, falls es noch nicht existiert
output_dir = "data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Exportiere den zusammengeführten Datensatz
merged_data.to_csv(f"{output_dir}/zusammengefuehrt.csv", index=False)

In [7]:
# **7. Vorschau auf den finalen Datensatz**
print("Zusammengeführter Datensatz (erste 5 Zeilen):")
print(merged_data.head())

Zusammengeführter Datensatz (erste 5 Zeilen):
       Datum  Warengruppe      Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0 2013-07-01            1  148.828353          0.0         6.0     17.8375   
1 2013-07-02            1  159.793757          0.0         3.0     17.3125   
2 2013-07-03            1  111.885594          0.0         7.0     21.0750   
3 2013-07-04            1  168.864941          0.0         7.0     18.8500   
4 2013-07-05            1  171.280754          0.0         5.0     19.9750   

   Windgeschwindigkeit  Wettercode Name  
0                 15.0        20.0    0  
1                 10.0         0.0    0  
2                  6.0        61.0    0  
3                  7.0        20.0    0  
4                 12.0         0.0    0  
